In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_0_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_csv/"+'gcc6'+"_0_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_0_32.head()

(2358543, 2)
reset_index 완료
input data shape


,hex,bin
0,55,1
1,89,0
2,e5,0
3,56,0
4,53,0


In [2]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['hex'].unique()))

# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot = pd.get_dummies(gcc6_0_32)


print('원핫인코딩완료')

print(gcc6_0_32_onehot.shape)

# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32_onehot['bin'].value_counts())

256
원핫인코딩완료
(2358543, 257)
0    2346253
1      12290
Name: bin, dtype: int64


In [3]:
# (5-1) gcc3 6gram
######################## 
idx3 = gcc6_0_32_onehot[gcc6_0_32_onehot['bin']==1].index  # 407, 474 ...
ls3 = list(idx3)

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 
left_idx3, right_idx3 = 0, 7 # 3개씩

# 6gram
for k in range(left_idx3, right_idx3):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능
print(len(ls_idx3))

#ls_idx3 = list(set(ls_idx3)) 
print(len(ls_idx3))

ls_idx3.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx3 = list(filter(lambda x: x<len(gcc6_0_32_onehot), ls_idx3))
print(len(ls_idx3))

# 2차 남은 index들 중 right_idx3 나눈 나머지 없애기
sub = len(ls_idx3)%(right_idx3)
print(sub)

ls_idx3 = ls_idx3[:len(ls_idx3)-sub]
print(len(ls_idx3))

print('gcc6_0_32', len(ls_idx3))

# loc 로 수정필요
gcc6_0_32_onehot_3gram = gcc6_0_32_onehot.loc[ls_idx3,:].copy()

86030
86030
86027
4
86023
gcc6_0_32 86023


In [54]:
# 훈련 데이터, 훈련 라벨
x_gcc6_0_32_3 = gcc6_0_32_onehot_3gram.iloc[:,1:].to_numpy()
y_gcc6_0_32_3 = gcc6_0_32_onehot_3gram['bin'].to_numpy()
print(x_gcc6_0_32_3.shape, y_gcc6_0_32_3.shape)


(86023, 256) (86023,)


In [55]:
x_gcc6_0_32_3 = x_gcc6_0_32_3.reshape(-1, right_idx3, x_gcc6_0_32_3.shape[1])
y_gcc6_0_32_3 = y_gcc6_0_32_3.reshape(-1, right_idx3, 1)

print(x_gcc6_0_32_3.shape, y_gcc6_0_32_3.shape)

(12289, 7, 256) (12289, 7, 1)


## 모델

In [57]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx3, 256)) 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 7, 256)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 7, 32)             34944     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 7, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________


In [58]:
########## 3gram

# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc6_0_32_3, y_gcc6_0_32_3):
    print('======Training stage======')
    model1.fit(x_gcc6_0_32_3[train],
               y_gcc6_0_32_3[train],
               epochs = 30,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc6_0_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

======Training stage======
Epoch 1/30
11060/11060 [==============================] - 2s 211us/step - loss: 0.1910 - accuracy: 0.9335
Epoch 2/30
11060/11060 [==============================] - 2s 162us/step - loss: 0.0195 - accuracy: 0.9948
Epoch 3/30
11060/11060 [==============================] - 2s 162us/step - loss: 0.0074 - accuracy: 0.9992
Epoch 4/30
11060/11060 [==============================] - 2s 161us/step - loss: 0.0044 - accuracy: 0.9994
Epoch 5/30
11060/11060 [==============================] - 2s 163us/step - loss: 0.0035 - accuracy: 0.9995
Epoch 6/30
11060/11060 [==============================] - 2s 160us/step - loss: 0.0033 - accuracy: 0.9995
Epoch 7/30
11060/11060 [==============================] - 2s 161us/step - loss: 0.0028 - accuracy: 0.9995
Epoch 8/30
11060/11060 [==============================] - 2s 160us/step - loss: 0.0026 - accuracy: 0.9995
Epoch 9/30
11060/11060 [==============================] - 2s 160us/step - loss: 0.0023 - accuracy: 0.9995
Epoch 10/30
11060/1

accuracy_score 0.999767522957108
recall_score 1.0
precision_score 0.9984779299847792
f1_score 0.9992383853769992
======Training stage======
Epoch 1/30
11060/11060 [==============================] - 2s 158us/step - loss: 6.7986e-04 - accuracy: 0.9999
Epoch 2/30
11060/11060 [==============================] - 2s 159us/step - loss: 6.1303e-04 - accuracy: 0.9998
Epoch 3/30
11060/11060 [==============================] - 2s 158us/step - loss: 5.8514e-04 - accuracy: 0.9999
Epoch 4/30
11060/11060 [==============================] - 2s 159us/step - loss: 5.3075e-04 - accuracy: 0.9999
Epoch 5/30
11060/11060 [==============================] - 2s 159us/step - loss: 5.2859e-04 - accuracy: 0.9999
Epoch 6/30
11060/11060 [==============================] - 2s 160us/step - loss: 5.7009e-04 - accuracy: 0.9999
Epoch 7/30
11060/11060 [==============================] - 2s 160us/step - loss: 5.2291e-04 - accuracy: 0.9999
Epoch 8/30
11060/11060 [==============================] - 2s 161us/step - loss: 4.7076e-04

accuracy_score 0.999767522957108
recall_score 0.9984423676012462
precision_score 1.0
f1_score 0.9992205767731879
======Training stage======
Epoch 1/30
11060/11060 [==============================] - 2s 160us/step - loss: 5.3459e-04 - accuracy: 0.9999
Epoch 2/30
11060/11060 [==============================] - 2s 157us/step - loss: 4.7467e-04 - accuracy: 0.9999
Epoch 3/30
11060/11060 [==============================] - 2s 158us/step - loss: 3.8740e-04 - accuracy: 0.9999
Epoch 4/30
11060/11060 [==============================] - 2s 157us/step - loss: 3.8074e-04 - accuracy: 0.9999
Epoch 5/30
11060/11060 [==============================] - 2s 156us/step - loss: 3.6916e-04 - accuracy: 0.9999
Epoch 6/30
11060/11060 [==============================] - 2s 158us/step - loss: 3.9782e-04 - accuracy: 0.9999
Epoch 7/30
11060/11060 [==============================] - 2s 158us/step - loss: 3.6775e-04 - accuracy: 0.9999
Epoch 8/30
11060/11060 [==============================] - 2s 158us/step - loss: 3.2304e-04

accuracy_score 0.999767522957108
recall_score 0.9984790874524715
precision_score 1.0
f1_score 0.9992389649923896
======Training stage======
Epoch 1/30
11060/11060 [==============================] - 2s 159us/step - loss: 2.6281e-04 - accuracy: 0.9999
Epoch 2/30
11060/11060 [==============================] - 2s 160us/step - loss: 2.7855e-04 - accuracy: 0.9999
Epoch 3/30
11060/11060 [==============================] - 2s 159us/step - loss: 1.5114e-04 - accuracy: 0.9999
Epoch 4/30
11060/11060 [==============================] - 2s 178us/step - loss: 1.1865e-04 - accuracy: 0.9999
Epoch 5/30
11060/11060 [==============================] - 2s 182us/step - loss: 5.7575e-05 - accuracy: 1.0000
Epoch 6/30
11060/11060 [==============================] - 2s 182us/step - loss: 4.6022e-05 - accuracy: 1.0000
Epoch 7/30
11060/11060 [==============================] - 2s 183us/step - loss: 8.8985e-05 - accuracy: 1.0000
Epoch 8/30
11060/11060 [==============================] - 2s 181us/step - loss: 4.1479e-05

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
11060/11060 [==============================] - 2s 180us/step - loss: 4.2924e-06 - accuracy: 1.0000
Epoch 2/30
11060/11060 [==============================] - 2s 178us/step - loss: 7.8176e-06 - accuracy: 1.0000
Epoch 3/30
11060/11060 [==============================] - 2s 181us/step - loss: 1.2580e-06 - accuracy: 1.0000
Epoch 4/30
11060/11060 [==============================] - 2s 183us/step - loss: 2.6837e-06 - accuracy: 1.0000
Epoch 5/30
11060/11060 [==============================] - 2s 181us/step - loss: 3.6945e-06 - accuracy: 1.0000
Epoch 6/30
11060/11060 [==============================] - 2s 179us/step - loss: 8.1009e-07 - accuracy: 1.0000
Epoch 7/30
11060/11060 [==============================] - 2s 178us/step - loss: 6.4152e-07 - accuracy: 1.0000
Epoch 8/30
11060/11060 [==============================] - 2s 182us/step - loss: 1.3446e-06 - accuracy: 1.0000
Epoch 9/30
11060/11060 [

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
11060/11060 [==============================] - 2s 177us/step - loss: 1.5690e-08 - accuracy: 1.0000
Epoch 2/30
11060/11060 [==============================] - 2s 176us/step - loss: 1.0575e-08 - accuracy: 1.0000
Epoch 3/30
11060/11060 [==============================] - 2s 180us/step - loss: 8.5799e-09 - accuracy: 1.0000
Epoch 4/30
11060/11060 [==============================] - 2s 179us/step - loss: 1.1729e-08 - accuracy: 1.0000
Epoch 5/30
11060/11060 [==============================] - 2s 178us/step - loss: 6.7905e-09 - accuracy: 1.0000
Epoch 6/30
11060/11060 [==============================] - 2s 179us/step - loss: 1.3108e-08 - accuracy: 1.0000
Epoch 7/30
11060/11060 [==============================] - 2s 179us/step - loss: 9.8822e-09 - accuracy: 1.0000
Epoch 8/30
11060/11060 [==============================] - 2s 178us/step - loss: 9.9397e-09 - accuracy: 1.0000
Epoch 9/30
11060/11060 [

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
11060/11060 [==============================] - 2s 177us/step - loss: 4.3215e-09 - accuracy: 1.0000
Epoch 2/30
11060/11060 [==============================] - 2s 177us/step - loss: 3.7009e-09 - accuracy: 1.0000
Epoch 3/30
11060/11060 [==============================] - 2s 176us/step - loss: 3.5023e-09 - accuracy: 1.0000
Epoch 4/30
11060/11060 [==============================] - 2s 175us/step - loss: 4.7319e-09 - accuracy: 1.0000
Epoch 5/30
11060/11060 [==============================] - 2s 177us/step - loss: 3.4083e-09 - accuracy: 1.0000
Epoch 6/30
11060/11060 [==============================] - 2s 176us/step - loss: 3.4807e-09 - accuracy: 1.0000
Epoch 7/30
11060/11060 [==============================] - 2s 176us/step - loss: 4.0635e-09 - accuracy: 1.0000
Epoch 8/30
11060/11060 [==============================] - 2s 175us/step - loss: 3.3498e-09 - accuracy: 1.0000
Epoch 9/30
11060/11060 [

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
11060/11060 [==============================] - 2s 173us/step - loss: 1.8146e-09 - accuracy: 1.0000
Epoch 2/30
11060/11060 [==============================] - 2s 173us/step - loss: 2.8148e-09 - accuracy: 1.0000
Epoch 3/30
11060/11060 [==============================] - 2s 175us/step - loss: 2.0017e-09 - accuracy: 1.0000
Epoch 4/30
11060/11060 [==============================] - 2s 175us/step - loss: 1.6999e-09 - accuracy: 1.0000
Epoch 5/30
11060/11060 [==============================] - 2s 175us/step - loss: 1.9209e-09 - accuracy: 1.0000
Epoch 6/30
11060/11060 [==============================] - 2s 173us/step - loss: 1.5975e-09 - accuracy: 1.0000
Epoch 7/30
11060/11060 [==============================] - 2s 174us/step - loss: 1.6938e-09 - accuracy: 1.0000
Epoch 8/30
11060/11060 [==============================] - 2s 175us/step - loss: 1.6214e-09 - accuracy: 1.0000
Epoch 9/30
11060/11060 [

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
11061/11061 [==============================] - 2s 175us/step - loss: 3.1024e-10 - accuracy: 1.0000
Epoch 2/30
11061/11061 [==============================] - 2s 174us/step - loss: 2.7406e-10 - accuracy: 1.0000
Epoch 3/30
11061/11061 [==============================] - 2s 174us/step - loss: 2.5096e-10 - accuracy: 1.0000
Epoch 4/30
11061/11061 [==============================] - 2s 174us/step - loss: 2.3479e-10 - accuracy: 1.0000
Epoch 5/30
11061/11061 [==============================] - 2s 172us/step - loss: 2.1786e-10 - accuracy: 1.0000
Epoch 6/30
11061/11061 [==============================] - 2s 169us/step - loss: 2.2094e-10 - accuracy: 1.0000
Epoch 7/30
11061/11061 [==============================] - 2s 171us/step - loss: 2.0631e-10 - accuracy: 1.0000
Epoch 8/30
11061/11061 [==============================] - 2s 171us/step - loss: 2.0554e-10 - accuracy: 1.0000
Epoch 9/30
11061/11061 [

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.999767522957108, 0.999767522957108, 0.999767522957108, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [1.0, 0.9984423676012462, 0.9984790874524715, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9984779299847792, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9992383853769992, 0.9992205767731879, 0.9992389649923896, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [59]:
# 4gram 평가지표
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999302568871326
10-Fold Cross_validation. Recall : 0.9996921455053718
10-Fold Cross_validation. Precision : 0.999847792998478
10-Fold Cross_validation. F1-Score : 0.9997697927142577
